# **Instalações e bibliotecas**

In [9]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import csv
import math
import random

####### Específicos para visualização #######

from pandas.plotting import scatter_matrix
from matplotlib import cm

####### Específicos para treinamento #######

from itertools import product

from sklearn.naive_bayes import GaussianNB
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.mixture import GaussianMixture




# **Funções proprietárias**

In [10]:
from pylab import rcParams
from scipy import stats


def plot_densities(data):

    rcParams['figure.figsize'] = 12, 20

    # Filtra os dados de acordo com as sáidas possíveis

    outcome_0 = data[data['Diag'] == 0]
    outcome_1 = data[data['Diag'] == 1]

    # Cria a figura 
    fig, axs = plt.subplots(8, 1)
    fig.suptitle('Features densities for different outcomes 0/1')
    plt.subplots_adjust(left = 0.25, right = 0.9, bottom = 0.1, top = 0.95,
                        wspace = 0.2, hspace = 0.9)

    # mostra as densidades de acordo com as variáveis de entrada
    
    for column_name in names[:-1]: 
        ax = axs[names.index(column_name)]
        #plt.subplot(4, 2, names.index(column_name) + 1)
        outcome_0[column_name].plot(kind='density', ax=ax, subplots=True, 
                                    sharex=False, color="blue", legend=True,
                                    label=column_name + ' Diagnóstico = 0')
        outcome_1[column_name].plot(kind='density', ax=ax, subplots=True, 
                                     sharex=False, color="black", legend=True,
                                     label=column_name + ' Diagnóstico = 1')
        
        KStest = stats.ks_2samp(outcome_0[column_name], outcome_1[column_name])

        ax.set_xlabel(column_name + ' valores')
        ax.set_title(column_name + ' Densidade')
        ax.grid('on')

    plt.show()
    fig.savefig('densities.png')

# **Base de dados**

1. Number of times pregnant 
2. Plasma glucose concentration a 2 hours in an oral glucose tolerance test
3. Diastolic blood pressure (mm Hg)
4. Triceps skin fold thickness (mm)
5. 2-Hour serum insulin (mu U/ml)
6. Body mass index (weight in kg/(height in m)^2)
7. Diabetes pedigree function
8. Age (years)
9. Class variable (0 or 1)

In [13]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#indexing-integer

df = pd.read_csv('/content/drive/My Drive/Pós-Graduação/Machine Learning - Métodos Probabilísticos/diabetes.csv')

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Exploração**

**Conteúdo sugerido**

1.   Matriz de correlação ordenada por significância
2.   Matriz de histogramas
3.   Matriz de densidade de probabildiada. Indentifique as principais prováveis distribuições de probabilidade " por inspeção".
4.   Mapa de calor indicando as principais correlações. o que você conclui?
5.   Matriz de espalhamento com a diagonal principal indicando as respectivas densidades. Explore um solormap que possibilite uma boa qualidade de visualização.
6.   Compare as densidades de probabildide para cada variável $X$ diferenciando se o diagnóstico é positivo ou negativo. Plote ambos em cores diferentes e sobrepostos visando estimar a grau de separabilidade.




In [14]:
df.head()

,NG,Glicose,PD,Triceps,Insulina,IMC,DP,Idade,Diag
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
corr_matrix = df.corr()
corr_matrix["Diag"].sort_values(ascending=False)

In [ ]:
df.hist(bins=10,figsize=(20, 10))
plt.show()

In [ ]:
df.plot(kind='density', subplots=True, layout=(4,3), sharex=False,figsize=(20, 10))
plt.show()

In [ ]:
sns.distplot(df['Diag'], hist=True, kde=True,
             bins='auto', color = 'darkblue',
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})
plt.show()

In [ ]:
colum_names = ['NG', 'Glicose', 'PD', 'Triceps', 'Insulina', 'IMC', 'DP', 'Idade','Diag']

In [ ]:
correlations = df.corr()

# Ajusta o tamanho da figura
fig, ax = plt.subplots(figsize=(10, 10))

# Gera o mapa de cor
colormap = sns.diverging_palette(220, 10, as_cmap=True)

# Permite a visualização dos valores
sns.heatmap(correlations, cmap=colormap, annot=True, fmt=".2f")
ax.set_xticklabels(
    colum_names,
    rotation=45,
    horizontalalignment='right'
);
ax.set_yticklabels(colum_names);
plt.show()

In [ ]:
y = df['Diag']

cmap = cm.get_cmap('viridis')

sm = scatter_matrix(df, figsize=(10, 10), diagonal='kde',c=y, cmap=cmap)

# Modifica o ângulo do label
[s.xaxis.label.set_rotation(40) for s in sm.reshape(-1)]
[s.yaxis.label.set_rotation(0) for s in sm.reshape(-1)]

# Ajusta o offset para não sopbrepor
[s.get_yaxis().set_label_coords(-0.6,0.5) for s in sm.reshape(-1)]

# Não mostra os "ticks"
[s.set_xticks(()) for s in sm.reshape(-1)]
[s.set_yticks(()) for s in sm.reshape(-1)]

plt.show()

# **Classificação**


**Conteúdo sugerido:**

1.   Compare os métodos: Naives Bayes, KNN e SVM. Monte a matriz de confusão, estime as principais métricas e explore a métrica Cohen-Kappa. Alguma vantagem acerca do método probabilístico? Qual pode ser a métrica mais indicada neste caso?
2.    Normalize os dados  e verifique se houve ganho em termos de desempenho. Comente os resultados.
3. Aplique PCA e trabalhe apenas com duas dimensões. Compare os métodos em temros de desempenho.



## **Preprocessing**

### Separando as variáveis dependentes e independentes

In [28]:
X = df.iloc[:-1,:-1]
X.head()


,NG,Glicose,PD,Triceps,Insulina,IMC,DP,Idade
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [ ]:
y = df.iloc[:-1,8]
y.head()

In [ ]:
# Carrega os dados em um novo data frame para manter o original

data  = df
names = list(data.columns)

# Visualizando as densidades de probabilidade separadas para cada saída esperada.

plot_densities(data)

## **Learning and validation**

### **Naive Bayes**

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)

In [ ]:
model = GaussianNB()
model.fit(X_train, y_train)

In [ ]:
expectedNB = y_test
predictedNB = model.predict(X_test)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(expectedNB, predictedNB))


In [ ]:
from sklearn.metrics import confusion_matrix
mat = confusion_matrix(predictedNB, expectedNB)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('Verdadeiro')
plt.ylabel('Predição');

In [ ]:
from sklearn.metrics import confusion_matrix, cohen_kappa_score

cohen_kappa_score(expectedNB, predictedNB)

In [ ]:
from sklearn.inspection import permutation_importance

imps = permutation_importance(model, X_test, y_test)
print(imps.importances_mean)

### **KNN**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=2)
neigh.fit(X_train, y_train)

In [ ]:
expectedKNN = y_test
predictedKNN = neigh.predict(X_test)

In [ ]:
print(metrics.classification_report(expectedKNN, predictedKNN))

In [ ]:
mat = confusion_matrix(predictedKNN, expectedKNN)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('Verdadeiro')
plt.ylabel('Predição');

In [ ]:
cohen_kappa_score(expectedKNN, predictedKNN)

In [ ]:
imps = permutation_importance(neigh, X_test, y_test)
print(imps.importances_mean)

In [ ]:
from sklearn.linear_model import LogisticRegression
LGR = LogisticRegression(solver='liblinear', random_state=0)
LGR.fit(X_train, y_train)

In [ ]:
LGR.predict_proba(X_train)

In [ ]:
predictedLGR = LGR.predict(X_test)
expectedLGR = y_test

In [ ]:
print(metrics.classification_report(expectedLGR, predictedLGR))

In [ ]:
mat = confusion_matrix(predictedKNN, expectedKNN)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('Verdadeiro')
plt.ylabel('Predição');

In [ ]:
cohen_kappa_score(expectedLGR, predictedLGR)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

sklearn_pca = PCA(n_components = 2)
Y_kmeans = sklearn_pca.fit_transform(X)

kmeans = KMeans(n_clusters=2, max_iter=600, algorithm = 'auto')
fitted = kmeans.fit(Y_kmeans)
prediction = kmeans.predict(Y_kmeans)

In [ ]:
# now let's try GMM clustering, which tries to fit normally-distributed clusters, 
# and might be the case when measuring things like petal and sepal sizes...

from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=3)
gmm.fit(X)

# predict the cluster for each data point
y_cluster_gmm = gmm.predict(X)
y_cluster_gmm

In [ ]:
cohen_kappa_score(y, y_cluster_gmm)

In [ ]:
y

### **SVM**

In [ ]:
from sklearn import svm

clf = svm.SVC()
clf.fit(X_train, y_train)

In [ ]:
expectedSVM = y_test
predictedSVM = clf.predict(X_test)

In [ ]:
print(metrics.classification_report(expectedSVM, predictedSVM))

In [ ]:
mat = confusion_matrix(predictedSVM, expectedSVM)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False)
plt.xlabel('Verdadeiro')
plt.ylabel('Predição');

In [ ]:
cohen_kappa_score(expectedSVM, predictedSVM)

In [ ]:
imps = permutation_importance(clf, X_test, y_test)
print(imps.importances_mean)

# **Agrupamento (cluster)**



1.   Utilize o método k-means para tentar agrupar os dados referentes ao dignóstico. Oberve que o método pode modificar a forma como é designado o diagnóstico. pesquise se há uma forma de impor.
2.  Utilize o método GMM para tentar agrupar os dados referentes ao dignóstico. Oberve que o método pode modificar a forma como é designado o diagnóstico. pesquise se há uma forma de impor.
3.  Elanore um método que possibilite otimizar o número de agrupmaneots possíveis e, valide se a proposta faz sentido ou não.



# **Visualização**


Utilizando dimensionalidade 2 (visando facilitar a visualização)

1.   Compare os classificadores utilizando um mapa de cores de fundo e dados, visando mostrar as fronteiras estabelecidas.
2.   Compare os agrupamentos utilizando um mapa de cores de fundo e dados, visando mostrar as fronteiras estabelecidas.



In [29]:
df.sample(3)

,NG,Glicose,PD,Triceps,Insulina,IMC,DP,Idade,Diag
34,10,122,78,31,0,27.6,0.512,45,0
584,8,124,76,24,600,28.7,0.687,52,1
687,1,107,50,19,0,28.3,0.181,29,0


In [30]:
X = np.array(df[['Glicose','Idade']])
y = np.array(df[['Diag']])
y = np.ravel(y,order='C')

In [ ]:
####### Criando os classificadores

clf1 = GaussianNB()
clf2 = KNeighborsClassifier(n_neighbors=2)
clf3 = SVC(gamma=.1, kernel='rbf', probability=True)

####### Treinando os calssificadores

clf1.fit(X, y)
clf2.fit(X, y)
clf3.fit(X, y)

####### Regiões de convergência
#
# Cria uma malha para predição do classificador em estudo

x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.1),
                     np.arange(y_min, y_max, 0.1))

f, axarr = plt.subplots(2, 2, sharex='col', sharey='row', figsize=(10, 8))

####### Para cada classificador faz uma predição e compara

for idx, clf, tt in zip(product([0, 1], [0, 1]),
                        [clf1, clf2, clf3],
                        ['Naive Bayes', 'KNN (k=2)',
                         'Kernel SVM']):

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    axarr[idx[0], idx[1]].contourf(xx, yy, Z, alpha=0.4)
    axarr[idx[0], idx[1]].scatter(X[:, 0], X[:, 1], c=y,
                                  s=20, edgecolor='k')
    axarr[idx[0], idx[1]].set_title(tt)

plt.show()

# **Referências**

In [ ]:
#############################################################################################################
# Code source: https://stackoverflow.com/questions/56153726/plot-k-nearest-neighbor-graph-with-8-features   #
#                                                                                                           #
#                                                                                                           #
# Modified for class use and explanation about Densities comparison by V.C.Parro                            #
#                                                                                                           #
# Data base: Diabetes                                                                                       #
# License: BSD 3 clause                                                                                     #
#                                                                                                           #
#############################################################################################################

https://scikit-learn.org/stable/auto_examples/ensemble/plot_voting_decision_regions.html